<a href="https://colab.research.google.com/github/OVP2023/NLP/blob/main/RAG_%D0%BC%D0%B5%D1%80%D1%82%D0%B2%D1%8B%D0%B5_%D0%B4%D1%83%D1%88%D0%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
#Загрузка текcтов локальной базы в RAG
frame_txt = pd.read_csv('/content/drive/MyDrive/Datasets/NLP/RAG_MD/Мертвые души.txt',sep='  ')
frame_txt

<ipython-input-6-afc5a693b92a>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  frame_txt = pd.read_csv('/content/drive/MyDrive/Datasets/NLP/RAG_MD/Мертвые души.txt',sep='  ')


,Мертвые души.
0,Том первый.
1,Глава I
2,В ворота гостиницы губернского города NN въеха...
3,"Когда экипаж въехал на двор, господин был встр..."
4,Пока приезжий господин осматривал свою комнату...
...,...
1168,"""Эхе-хе! что ж ты?"" сказал Чичиков Селифану: ""..."
1169,"""Что?"" сказал Селифан медленным голосом."
1170,"""Как что? Гусь ты! как ты едешь? Ну же, потрог..."
1171,"И в самом деле Селифан давно уже ехал, зажмуря..."


In [ ]:
frame_txt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Мертвые души.  1173 non-null   object
dtypes: object(1)
memory usage: 9.3+ KB


In [ ]:
frame_txt["Мертвые души."][5]

'Покамест слуги управлялись и возились, господин отправился в общую залу. Какие бывают эти общие залы — всякой проезжающий знает очень хорошо: те же стены, выкрашенные масляной краской, потемневшие вверху от трубочного дыма и залосненные снизу спинами разных проезжающих, а еще более туземными купеческими, ибо купцы по торговым дням приходили сюда сам-шест и сам-сём испивать свою известную пару чаю; тот же закопченный потолок, та же копченая люстра со множеством висящих стеклышек, которые прыгали и звенели всякий раз, когда половой бегал по истертым клеенкам, помахивая бойко подносом, на котором сидела такая же бездна чайных чашек, как птиц на морском берегу; те же картины во всю стену, писанные масляными красками; словом, всё то же, что и везде; только и разницы, что на одной картине изображена была нимфа с такими огромными грудями, каких читатель, верно, никогда не видывал. Подобная игра природы, впрочем, случается на разных исторических картинах, неизвестно в какое время, откуда и ке

In [10]:
#Деление на чанки
!pip install langchain

In [ ]:
#в колабе уже установлена nltk
#удалим стоп слова из текста

In [15]:
import nltk
from nltk.corpus import stopwords

In [24]:
nltk.download('stopwords')
mystopwords = stopwords.words('russian') + ['\n'] + ['—']

def  remove_stopwords(text, mystopwords=mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

frame_txt["Мертвые души."]= frame_txt["Мертвые души."].apply(remove_stopwords)
frame_txt["Мертвые души."][2]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка, ездят холостяки: отставные подполковники, штабс-капитаны, помещики, имеющие около сотни душ крестьян, словом, те, которых называют господами средней руки. В бричке сидел господин, красавец, дурной наружности, слишком толст, слишком тонок; сказать, стар, однако так, слишком молод. Въезд произвел городе совершенно никакого шума сопровожден ничем особенным; русские мужика, стоявшие дверей кабака против гостиницы, сделали кое-какие замечания, относившиеся, впрочем, экипажу, сидевшему нем. "Вишь ты", сказал другому, "вон какое колесо! Что думаешь, доедет колесо, б случилось Москву, доедет?" "Доедет", отвечал другой. "А Казань-то, думаю, доедет?" "В Казань доедет", отвечал другой. Этим разговор кончился. Да еще, бричка подъехала гостинице, встретился молодой человек белых канифасовых панталонах, весьма узких коротких, фраке покушеньями моду, из-под которого видна манишка, застегнутая тульскою була

In [11]:
from langchain.text_splitter import CharacterTextSplitter

In [25]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=500,
    chunk_overlap=50
)
texts = text_splitter.split_text("\n".join(frame_txt["Мертвые души."]))

In [26]:
texts

['Том первый.\nГлава I',
 'В ворота гостиницы губернского города NN въехала довольно красивая рессорная небольшая бричка, ездят холостяки: отставные подполковники, штабс-капитаны, помещики, имеющие около сотни душ крестьян, словом, те, которых называют господами средней руки. В бричке сидел господин, красавец, дурной наружности, слишком толст, слишком тонок; сказать, стар, однако так, слишком молод. Въезд произвел городе совершенно никакого шума сопровожден ничем особенным; русские мужика, стоявшие дверей кабака против гостиницы, сделали кое-какие замечания, относившиеся, впрочем, экипажу, сидевшему нем. "Вишь ты", сказал другому, "вон какое колесо! Что думаешь, доедет колесо, б случилось Москву, доедет?" "Доедет", отвечал другой. "А Казань-то, думаю, доедет?" "В Казань доедет", отвечал другой. Этим разговор кончился. Да еще, бричка подъехала гостинице, встретился молодой человек белых канифасовых панталонах, весьма узких коротких, фраке покушеньями моду, из-под которого видна манишка,